In [4]:
start_date='2020-07-11'
end_date='2023-07-11'
geometry=(92.76,26.54)

In [ ]:
import ee
import geemap
import pandas as pd



# Initialize Earth Engine
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except Exception:
    ee.Authenticate()
    ee.Initialize()

print("Earth Engine initialized successfully")


Earth Engine initialized successfully


In [6]:
ee_geometry = ee.Geometry.Point(geometry)


In [7]:
def get_dates_s1(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    return ee.Feature(None, {'date': date})

s1 = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterDate(start_date, end_date)
    .filterBounds(ee_geometry)
)

fc_s1 = s1.map(get_dates_s1).getInfo()



In [8]:
def get_dates_s2(image):
    date = image.date().format('YYYY-MM-dd')
    return ee.Feature(None, {'date': date})

s2 = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .filterDate(start_date, end_date)
    .filterBounds(ee_geometry)
)

fc_s2 = s2.map(get_dates_s2).getInfo()


In [9]:
df_s1 = pd.DataFrame(
    [f['properties']['date'] for f in fc_s1['features']],
    columns=['Date']
)

df_s2 = pd.DataFrame(
    [f['properties']['date'] for f in fc_s2['features']],
    columns=['Date']
)

df_s1['Date'] = pd.to_datetime(df_s1['Date'])
df_s2['Date'] = pd.to_datetime(df_s2['Date'])


In [10]:
paired_dates = pd.merge(df_s1, df_s2, on='Date', how='inner')
paired_dates = paired_dates.sort_values('Date').reset_index(drop=True)

paired_dates.head()


,Date
0,2020-09-20
1,2020-11-07
2,2020-11-12
3,2020-11-24
4,2020-11-29


In [20]:
# Convert paired dates to list
date_list = paired_dates['Date'].dt.strftime('%Y-%m-%d').tolist()

# Example: choose one flood event
post_date = date_list[-1]          # latest date
pre_date = date_list[-2]           # just before flood

pre_date, post_date


('2023-05-08', '2023-07-05')

In [21]:
# Sentinel-1 collection
s1 = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterBounds(ee_geometry)
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
)

# Pre-flood image
s1_pre = (
    s1.filterDate(
        ee.Date(pre_date).advance(-3, 'day'),
        ee.Date(pre_date).advance(3, 'day')
    )
    .median()
    .select('VV')
)

# Post-flood image
s1_post = (
    s1.filterDate(
        ee.Date(post_date).advance(-3, 'day'),
        ee.Date(post_date).advance(3, 'day')
    )
    .median()
    .select('VV')
)

s1_pre, s1_post
 


(<ee.image.Image at 0x2344be616c0>, <ee.image.Image at 0x2344be61060>)

In [22]:
Map = geemap.Map(center=[geometry[1], geometry[0]], zoom=9)

Map.addLayer(s1_pre, {'min': -25, 'max': 0}, 'S1 Pre-Flood')
Map.addLayer(s1_post, {'min': -25, 'max': 0}, 'S1 Post-Flood')

Map


Map(center=[26.54, 92.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [23]:
import os
os.makedirs("data", exist_ok=True)

geemap.ee_export_image(
    s1_pre,
    filename="data/S1_pre.tif",
    scale=10,
    region=ee_geometry,
    file_per_band=False
)

geemap.ee_export_image(
    s1_post,
    filename="data/S1_post.tif",
    scale=10,
    region=ee_geometry,
    file_per_band=False
)


Generating URL ...
Please wait ...
Data downloaded to d:\minor project 1st ver\DeepSARFlood\data\S1_pre.tif
Generating URL ...
Please wait ...
Data downloaded to d:\minor project 1st ver\DeepSARFlood\data\S1_post.tif
